In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import h5py
import os
from tqdm import tqdm

from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

Set up

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

MEDIAN=False
AVG=True

figloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/lunchtalks'


# Single Source hc Map

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )
hc_ss = data[10]['hc_ss']
hc_bg = data[10]['hc_bg']
gamma_ssi = dsdat[10]['gamma_ssi']

In [ ]:


fig, ax = plot.figax_single(xlabel=plot.LABEL_GW_FREQUENCY_YR, 
                     )
ax.set_ylabel('Characteristic Strain $h_c$')
xx = data[10]['fobs_cents'] *YR
yy = xx**(-2/3)*1.1*10**-15

_, bg = holo.single_sources.resample_loudest(hc_ss, hc_bg, 1)
ax.plot(xx,yy, color='k', alpha=0.25, linestyle='dashed')

ax.plot(xx, np.median(bg, axis=-1), color='k')
for pp in [50,95]:
    lo, hi = np.percentile(bg, (50-pp/2, 50+pp/2), axis=-1)
    ax.fill_between(xx, lo, hi, color='k', alpha=0.2)


# ax.scatter(xx, np.max(hc_ss[:,0,:], axis=-1), color='k', alpha=0.2)

nreals = 5
nloudest = 3
cmap = plot.truncate_colormap('jet_r',0.1, 0.999)
colors = cmap(np.linspace(0,1,nreals))
colors = ['r',  'dodgerblue',
          'purple', 'g', 'orange']

seed = 7197
# seed = 89118
seed = 1811
# seed=None
if seed is None:
    seed = np.random.randint(99999)   # get a random number
    print(f"random seed: {seed}")                           # print it out so we can reuse it if desired
np.random.seed(seed)   

rands = np.random.choice(np.arange(0,len(hc_ss[0,0])), 
                         size=nreals, replace=False, )
for rr, rand in enumerate(rands):
    cc = colors[rr]
    ax.plot(xx, bg[:,rand], alpha=0.5, zorder=6)
    ax.scatter(xx, hc_ss[:,rand,0], marker='o', color=cc, alpha=0.2,
               edgecolors='k', zorder=5)
    for ll in range(nloudest):
        ax.scatter(xx, hc_ss[:,rand,ll], marker='o', color=cc, alpha=0.2,
                   )
        
name = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/lunchtalks'
name += f'/hc_{seed}.png'
fig.savefig(name,
            transparent=True, bbox_inches='tight', dpi=100)

In [ ]:


fig, ax = plot.figax(xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN,
                     figsize=(6.5,4))
xx = data[10]['fobs_cents'] *YR
yy = xx**(-2/3)*1.1*10**-15

_, bg = holo.single_sources.resample_loudest(hc_ss, hc_bg, 1)
ax.plot(xx,yy, color='k', alpha=0.25, linestyle='dashed')

ax.plot(xx, np.median(bg, axis=-1), color='k')
for pp in [50,95]:
    lo, hi = np.percentile(bg, (50-pp/2, 50+pp/2), axis=-1)
    ax.fill_between(xx, lo, hi, color='k', alpha=0.2)


# ax.scatter(xx, np.max(hc_ss[:,0,:], axis=-1), color='k', alpha=0.2)

nreals = 5
nloudest = 3
cmap = plot.truncate_colormap('jet_r',0.1, 0.999)
colors = cmap(np.linspace(0,1,nreals))
colors = ['r',  'dodgerblue',
          'purple', 'g', 'orange']

seed = 7197
# seed = 89118
seed = 1811
# seed=None
if seed is None:
    seed = np.random.randint(99999)   # get a random number
    print(f"random seed: {seed}")                           # print it out so we can reuse it if desired
np.random.seed(seed)   

rands = np.random.choice(np.arange(0,len(hc_ss[0,0])), 
                         size=nreals, replace=False, )
for rr, rand in enumerate(rands):
    cc = colors[rr]
    ax.plot(xx, bg[:,rand], alpha=0.5, zorder=6)
    ax.scatter(xx, hc_ss[:,rand,0], marker='o', color=cc, alpha=0.2,
               edgecolors='k', zorder=5)
    for ll in range(nloudest):
        ax.scatter(xx, hc_ss[:,rand,ll], marker='o', color=cc, alpha=0.2,
                   )
        
name = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/lunchtalks'
name += f'/hc_{seed}.png'
fig.savefig(name,
            transparent=True, bbox_inches='tight', dpi=100)

# Varying h_c and Mass

In [ ]:
cmap_Blues = plot.truncate_colormap('Blues', 0.4, 1)
cmap_Greens = plot.truncate_colormap('Greens', 0.4, 1)
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)
cmap_Greys = plot.truncate_colormap('Greys', 0.4, 0.9)

greys = cmap_Greys(np.linspace(0,1,3))

In [ ]:
maps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10',
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner'
]

In [ ]:

for tt, target in enumerate(targets):
    # if tt != 3:
    #     continue
    colors=maps[tt](np.linspace(0,1,3))
    fontsize=16

    xlabel=plot.LABEL_GW_FREQUENCY_YR
    ylabels = ['$h_c$', '$M\ [\mathrm{M}_\odot]$']
    xx, yy_ss, yy_bg, parlabels = detstats.get_hcpar_arrays(
                target, nloudest=1, parvars=[0,2,4],
            )

    for ii in [1,0]:
        fig, ax = plot.figax(figsize=(4,2.5))
        if ii==0: ax.set_xlabel(xlabel, fontsize=fontsize)
        ax.set_ylabel(ylabels[ii], fontsize=fontsize)
        handles = []
        labels = []
        for vv in range(3):
            ax.plot(xx, np.median(yy_bg[vv,ii,:,:], axis=-1), color=greys[vv],
                        linestyle='-', alpha=0.85, zorder=2)
            pp=68
            med, *conf = np.percentile(yy_ss[vv,ii,:,:], (50, 50-pp/2, 50+pp/2), axis=-1)
            ax.plot(xx, med, color=colors[vv], linestyle='--', alpha=0.75, zorder=3)
            ax.fill_between(xx, *conf, color=colors[vv], alpha=0.2, zorder=1)
            handles.append(mpl.lines.Line2D([0], [0], color=colors[vv]))
            labels.append(parlabels[vv])
            
        
        fig.patch.set_facecolor('white')
        fig.patch.set_alpha(0.2)
        fig.savefig(figloc+f'/{target}_par_{ii}.png', transparent=False, bbox_inches='tight')

        han = [
            mpl.lines.Line2D([0], [0], linestyle='--', label='SS',
                            color=colors[1]),
            mpl.lines.Line2D([0], [0], linestyle='-', label='BG',
                            color=greys[1])
        ]
        ax.legend(handles=han, loc='lower left')
    fig,ax = plot.figax_single(height=2)
    ax.legend(handles=handles, labels=labels, ncols=3, title=plot.PARAM_KEYS[target],
            fancybox=True, framealpha=0.6, loc='center')
    fig.savefig(figloc+f'/{target}_par_leg.png', transparent=True, bbox_inches='tight')


# EVSS

In [ ]:
col0p0 = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]

col_gw = [
    '#595959', '#595959',
    '#595959', '#595959',
    '#595959', '#595959', 
]

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10',
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner'
]

In [ ]:
for tt, target in enumerate(targets):
    if True:
        fig,ax = plot.figax(xscale='linear', figsize=(5,3.5))
        ax.set_xlabel(plot.PARAM_KEYS[target], fontsize=18)
        ax.set_ylabel(plot.LABEL_EVSS, fontsize=18)

        xx, yy = detstats.get_ratio_arrays(target, bgl=1)
        yy *= 0.5

        for pp in [50, 95]:
            med, *conf = np.percentile(yy, [50, 50-pp/2, 50+pp/2], axis=(1,2))
            ax.plot(xx, med, alpha=0.9, color=col0p0[tt])
            ax.fill_between(xx, *conf, color=col0p0[tt], alpha=0.25)


        fig.patch.set_facecolor('white')
        fig.patch.set_alpha(0.2)


        filename = figloc+f'/{target}_evss.png'
        fig.savefig(filename, transparent=False, facecolor=fig.get_facecolor(),
                    bbox_inches='tight')

# Schechter Function

In [ ]:

mass = np.logspace(6, 13, 91)*MSOL
colors= cmap_Greens(np.linspace(0,1,3))


fig,ax = plot.figax(figsize=(4,2.5))
ax.set_xlabel('$M\ [\mathrm{M}_\odot]$', fontsize=16)
ax.set_ylabel('$\Psi \equiv \\frac{d \eta_*}{d\log_{10} m_*}$', fontsize=20)

xx = mass/MSOL

handles = []
labels = []
for ii, mchar0_log10 in enumerate([10.5, 11.5, 12.5]):
    gsmf = holo.sams.comps.GSMF_Schechter(phi0=-2.5, mchar0_log10=mchar0_log10)
    phi = gsmf(mass, redz=1)
    hh, = ax.plot(xx, phi, color=colors[ii], alpha=0.9)
    handles.append(hh)

    ax.axvline(10**mchar0_log10, linestyle='--', color=colors[ii], alpha=0.5)
    labels.append(f'{mchar0_log10}')

ax.set_ylim(10**-8, 10**0)
ax.legend(handles=handles, labels=labels, title=plot.PARAM_KEYS['gsmf_mchar0_log10'],
          loc='lower left', ncols=1, fancybox=True, framealpha=0.2,)
fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.0)

filename = figloc+'/gsmf_func_mchar0.png'
fig.savefig(filename, transparent=False, bbox_inches='tight')

In [ ]:

mass = np.logspace(6, 13, 91)*MSOL
colors= cmap_Greens(np.linspace(0,1,3))


fig,ax = plot.figax(figsize=(4,2.5))
ax.set_xlabel('$M\ [\mathrm{M}_\odot]$', fontsize=16)
ax.set_ylabel('$\Psi \equiv \\frac{d \eta_*}{d\log_{10} m_*}$', fontsize=20)

xx = mass/MSOL

handles = []
labels = []
for ii, phi0 in enumerate([-3.5, -2.5, -1.5]):
    gsmf = holo.sams.comps.GSMF_Schechter(phi0=phi0, mchar0_log10=11.5)
    phi = gsmf(mass, redz=1)
    hh, = ax.plot(xx, phi, color=colors[ii], alpha=0.9)
    handles.append(hh)

    ax.axhline(10**phi0, linestyle='--', color=colors[ii], alpha=0.5)
    labels.append(f'{phi0}')

ax.set_ylim(10**-8, 10**0)
ax.legend(handles=handles, labels=labels, title=plot.PARAM_KEYS['gsmf_phi0'],
          loc='lower left', ncols=3, fancybox=True, framealpha=0.2,)

fig.patch.set_facecolor('white')
fig.patch.set_alpha(0.2)

filename = figloc+'/gsmf_func_phi0.png'
fig.savefig(filename, transparent=False, bbox_inches='tight')

# Separation Evolution